<a href="https://colab.research.google.com/github/SkyManiac09/bigdata_course/blob/main/BigData_IMDB_LLMforCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mounting Data
Mount stored imdb data in GDrive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls "/content/drive/My Drive/bigdata_imdb"

directing.json	 train-1.csv  train-3.csv  train-5.csv	train-7.csv  validation_hidden.csv
test_hidden.csv  train-2.csv  train-4.csv  train-6.csv	train-8.csv  writing.json


#### Data preparation
Load training data \
Planned space for data manipulation

In [5]:
import pandas as pd
import json
# Load train-x.csv files
#train_files = [f"../content/drive/My Drive/bigdata_imdb/train-{i}.csv" for i in range(1,9)]
train_files = ["../content/drive/My Drive/bigdata_imdb/train-1.csv"]
df_list = [pd.read_csv(file) for file in train_files]
df = pd.concat(df_list, ignore_index=True) #963 entries

In [6]:
# Get json data and prepare merging

with open('../content/drive/MyDrive/bigdata_imdb/writing.json', 'r') as f:
    writing_data = json.load(f)
writing_df = pd.json_normalize(writing_data)

with open('/content/drive/MyDrive/bigdata_imdb/directing.json','r') as f:
    directing_data = json.load(f)


In [35]:
print(writing_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22428 entries, 0 to 22427
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   movie   22428 non-null  object
 1   writer  22428 non-null  object
dtypes: object(2)
memory usage: 350.6+ KB
None


In [ ]:
# Extract the "movie" and "director" dictionaries
movie_dict = directing_data['movie']
director_dict = directing_data['director']

#print(movie_dict)
#print(list(movie_dict)[-1], movie_dict[list(movie_dict)[-1]]) #check last key:value pair
#print(director_dict)
#print(list(movie_dict)[-1], movie_dict[list(movie_dict)[-1]]) #check last key:value pair

# Create a DataFrame from the dictionaries
directing_df = pd.DataFrame({
    'tconst': list(movie_dict.values()),  # Movie IDs
    'director_nconst': list(director_dict.values())  # Director IDs
})

# Display the resulting DataFrame
print(directing_df.head())
directing_df.to_csv('/content/drive/MyDrive/bigdata_imdb/directing_df.csv', index=False)

In [ ]:
print(directing_df.info())

In [ ]:
print(writing_df.head())
writing_df.to_csv('/content/drive/MyDrive/bigdata_imdb/writing_df.csv', index=False)

In [30]:
# Merge writing_df and directing_df on 'movie' and 'tconst'
merged_df = pd.merge(writing_df, directing_df, left_on='movie', right_on='tconst', how='outer')
merged_df.drop(columns=['movie'], inplace=True)

# Save the transformed and merge data in csv
merged_df.to_csv('/content/drive/MyDrive/bigdata_imdb/writ_and_direct.csv', index=False)


In [ ]:
print(merged_df.info())

In [48]:
# Merge all writing and directing information to our training data
full_df = pd.merge(df, merged_df, on='tconst', how='left')

In [ ]:
print(df.info())

In [ ]:
print(full_df.info())
print(full_df.describe())
print(full_df.head())

#### Check Duplicates
Given that df has 963 entries \
merge_df has 27888 entries for all the tconst in this datasets \
But after left join, this results in 2488 entries in full_df

In [43]:
merged_df_duplicates = merged_df[merged_df.duplicated(subset=['tconst'], keep=False)]
print(merged_df_duplicates)

          writer     tconst director_nconst
0      nm0195339  tt0003740       nm0665163
1      nm0515385  tt0003740       nm0665163
2      nm0665163  tt0003740       nm0665163
3      nm0758215  tt0003740       nm0665163
4      nm0406585  tt0008663       nm0803705
...          ...        ...             ...
27883  nm0942647  tt9904802       nm0052054
27884  nm3853396  tt9904802       nm0052054
27885  nm2063122  tt9911196       nm0631590
27886  nm0277932  tt9911196       nm0631590
27887  nm3547655  tt9911196       nm0631590

[24437 rows x 3 columns]


In [44]:
df_duplicates = df[df.duplicated(subset=['tconst'], keep=False)]
print(df_duplicates)

Empty DataFrame
Columns: [Unnamed: 0, tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes, label]
Index: []


>>> One movie has many directors/writers. We'll have to decide how to deal with this.


>>> For the sake of simplicity, I'll just drop dupicate for now.

In [50]:
full_df.drop_duplicates(subset=['tconst'], keep='first', inplace=True) # Replace the first 'full_df' with the one that drops duplicate


## Working Data Modification
Since it takes around 1 hr to generate outputs of ~ 2200+ entries, I'll just do a snippet instead.

In [51]:
working_df = full_df[full_df.isnull().any(axis=1)]

In [54]:
print(working_df.info())
print(working_df[['primaryTitle','originalTitle']].head())

<class 'pandas.core.frame.DataFrame'>
Index: 549 entries, 59 to 2487
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       549 non-null    int64  
 1   tconst           549 non-null    object 
 2   primaryTitle     549 non-null    object 
 3   originalTitle    43 non-null     object 
 4   startYear        549 non-null    object 
 5   endYear          549 non-null    object 
 6   runtimeMinutes   549 non-null    object 
 7   numVotes         460 non-null    float64
 8   label            549 non-null    bool   
 9   writer           549 non-null    object 
 10  director_nconst  549 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(8)
memory usage: 47.7+ KB
None
                   primaryTitle               originalTitle
59         Juno and the Paycock                         NaN
62       Thé Éáglé ánd thé Háwk                         NaN
68          Émplớyéés' Éntráncé            

In [55]:
missing_counts = working_df.isnull().sum()
print(missing_counts)

Unnamed: 0           0
tconst               0
primaryTitle         0
originalTitle      506
startYear            0
endYear              0
runtimeMinutes       0
numVotes            89
label                0
writer               0
director_nconst      0
dtype: int64


In [57]:
#make a sample data for LLM task
testDI = working_df[working_df['originalTitle'].isnull()].sample(n=20)

## Initiate LLM model
Using Huggingface framework for model utilization

In [58]:
!pip install transformers accelerate
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [59]:
# Install Gemma-3
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3


  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-hbhfsvke
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-hbhfsvke
  Running command git checkout -q 1c0f782fe5f983727ff245c4c1b3906f9b99eec2
  Resolved https://github.com/huggingface/transformers to commit 1c0f782fe5f983727ff245c4c1b3906f9b99eec2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936429 sha256=665b7580e65533809d28b62fbb67fd781ecdf8857be20d38fbfdbb3f0f5cdf9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-p01xrp6d/wheels/53/15/d5/d63b866c641d8863f9cd29a4cc7a5efc38476c3aae8247c195
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transf

In [60]:
# Instantiate Model

from transformers import AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
import torch

model_id = "google/gemma-3-1b-it"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Gemma3ForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto") # Gemma3 specific

# Generalize code for other models
#model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Move the model to GPU
#model = model.to("cuda")

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# Define response generation function

def generate_response(prompt, max_tokens=100):
    """Generate LLM output based on a given prompt."""
    inputs = tokenizer(prompt, return_tensors="pt") #.to("cuda") >> run out of GPU usage
    output = model.generate(**inputs, max_new_tokens=max_tokens)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Data Imputation
def data_imputation(row, missing_column):
    """Fill missing values using the LLM."""
    prompt = f"""
    You are a database engineer. Fill in the missing value.

    Record:
    {row.to_dict()}

    What is the best guess for '{missing_column}'?
    """
    return generate_response(prompt)

# Example: Process a dataset
missing_col = "originalTitle"  # Change based on dataset

testDI[missing_col] = testDI.apply(lambda row: data_imputation(row, missing_col) if pd.isnull(row[missing_col]) else row[missing_col], axis=1)

# Save results
df.to_csv("preprocessed_data.csv", index=False)

### Areas to explore
Few-shot examples to make the generated output more on point

In [ ]:
# Few-shot examples (adjust based on dataset)
FEW_SHOT_EXAMPLES = """
You are a database engineer. Fill in missing values for the "city" column.

Examples:
Record: {'name': 'Joe’s Diner', 'addr': '123 Main St', 'phone': '212-555-7890', 'type': 'Restaurant', 'city': ???}
Reason: The phone number '212' suggests it is in New York City.
Answer: New York City

Record: {'name': 'The Green Leaf', 'addr': '45 Maple Rd', 'phone': '415-444-1234', 'type': 'Vegan', 'city': ???}
Reason: The phone number '415' is from San Francisco, California.
Answer: San Francisco

Now complete the missing city for the following records:
"""


Batch Processing

In [ ]:
def generate_response(prompt, max_tokens=100):
    """Generate a response from the LLM."""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=max_tokens)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def batch_imputation(records):
    """Processes multiple records in one LLM call (batch prompting)."""
    prompt = FEW_SHOT_EXAMPLES  # Start with few-shot examples
    for row in records:
        prompt += f"\nRecord: {row}\nReason:"

    response = generate_response(prompt)

    # Extract the generated answers (assumes they appear after 'Answer:' in the LLM output)
    answers = response.split("Answer: ")[1:]  # Splitting the response by "Answer:"
    return [answer.strip().split("\n")[0] for answer in answers]  # Take only first line per answer

# Process data in batches (e.g., 5 rows at a time)
batch_size = 5
missing_col = "city"
missing_rows = df[df[missing_col].isnull()].to_dict(orient="records")  # Convert missing rows to dictionary

for i in range(0, len(missing_rows), batch_size):
    batch = missing_rows[i : i + batch_size]  # Select batch
    imputed_values = batch_imputation(batch)  # Get imputed values

    # Fill missing values in the dataframe
    for j, row in enumerate(batch):
        df.loc[df.index == row["index"], missing_col] = imputed_values[j]

# Save the preprocessed data
df.to_csv("preprocessed_data.csv", index=False)
